<a href="https://colab.research.google.com/github/soudagarminhaz/learning-game/blob/main/LearningGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from collections import defaultdict

# ===== Enhanced Game Data =====
questions = [
    {
        "type": "question",
        "question": "What is the capital of France?",
        "options": ["Berlin", "Madrid", "Paris", "Rome"],
        "correct_answer": "Paris",
        "level": "easy",
        "explanation": "Paris has been the capital of France since 508 AD."
    },
    {
        "type": "question",
        "question": "Who painted the Mona Lisa?",
        "options": ["Van Gogh", "Da Vinci", "Picasso", "Monet"],
        "correct_answer": "Da Vinci",
        "level": "easy",
        "explanation": "Leonardo da Vinci painted it between 1503-1519."
    },
    {
        "type": "puzzle",
        "puzzle": "I speak without a mouth and hear without ears. What am I?",
        "answer": "echo",
        "level": "medium",
        "explanation": "An echo repeats sounds but has no physical form."
    },
    {
        "type": "puzzle",
        "puzzle": "What has keys but can't open locks?",
        "answer": "piano",
        "level": "hard",
        "explanation": "A piano has musical keys but can't open doors."
    }
]

stories = {
    "sun": "The golden sun rose majestically, casting its warm glow across the verdant landscape. Birds began their morning chorus as the world awakened to a new day full of possibilities.",
    "moon": "The silver moon hung low in the inky sky, its pale light illuminating the nocturnal world below. Crickets chirped their evening song as fireflies danced in the moonlight.",
    "ocean": "Endless waves crashed rhythmically against the weathered cliffs, their salty spray carried by the ocean breeze. Somewhere in the deep blue expanse, whales sang their ancient songs."
}

# ===== Improved Game State =====
class GameState:
    def __init__(self):
        self.reset()

    def reset(self):
        self.current_question = 0
        self.score = 0
        self.game_state = "setup"
        self.answered_questions = defaultdict(bool)
        self.filtered_questions = []
        self.progress_bar_max = len(questions)

game = GameState()

# ===== Enhanced Widgets =====
level_radio = widgets.RadioButtons(
    options=['easy', 'medium', 'hard'],
    description='Level:',
    layout={'width': 'max-content'}
)

type_radio = widgets.RadioButtons(
    options=['quiz', 'puzzle', 'story'],
    description='Type:',
    layout={'width': 'max-content'}
)

word_input = widgets.Text(
    placeholder='Try: sun, moon, ocean...',
    description='Word:',
    disabled=False
)

start_button = widgets.Button(
    description="Start Game",
    button_style='success',
    icon='play'
)

next_button = widgets.Button(
    description="Next ➔",
    button_style='info'
)

restart_button = widgets.Button(
    description="Play Again",
    button_style='warning'
)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=game.progress_bar_max,
    description='Progress:',
    bar_style='info'
)

output = widgets.Output()

# ===== Optimized Event Handlers =====
def filter_questions():
    game.filtered_questions = [
        q for q in questions
        if q['level'] == level_radio.value and
        (type_radio.value == 'quiz' and q['type'] == 'question' or
         type_radio.value == 'puzzle' and q['type'] == 'puzzle')
    ]
    random.shuffle(game.filtered_questions)
    progress_bar.max = len(game.filtered_questions)

def start_game(b):
    game.reset()
    if type_radio.value in ['quiz', 'puzzle']:
        filter_questions()
        if not game.filtered_questions:
            with output:
                clear_output()
                display(widgets.HTML("<h3 style='color:red'>No questions available for these filters!</h3>"))
            return
    game.game_state = "playing"
    update_display()

def handle_answer(btn):
    check_answer(btn.description)

def check_answer(answer):
    question = game.filtered_questions[game.current_question]

    if question["type"] == "question":
        correct = (answer == question["correct_answer"])
    else:
        correct = (answer.lower() == question["answer"].lower())

    if correct and not game.answered_questions[game.current_question]:
        game.score += 1
        game.answered_questions[game.current_question] = True

    game.game_state = "feedback"
    update_display()

def next_question(b):
    game.current_question += 1
    if game.current_question >= len(game.filtered_questions):
        game.game_state = "completed"
    else:
        game.game_state = "playing"
    update_display()

def restart_game(b):
    game.reset()
    progress_bar.value = 0
    update_display()

# ===== Refactored Display Logic =====
def update_display():
    progress_bar.value = game.current_question

    with output:
        clear_output(wait=True)

        if game.game_state == "setup":
            display(widgets.VBox([
                widgets.HTML("<h1 style='text-align:center'>🎮 GenAI Learning Game</h1>"),
                widgets.HBox([
                    level_radio,
                    type_radio
                ]),
                word_input if type_radio.value == "story" else widgets.HTML(),
                start_button
            ]))

        elif game.game_state == "playing":
            if type_radio.value == "story":
                word = word_input.value.strip().lower()
                if word in stories:
                    display(widgets.VBox([
                        widgets.HTML(f"<h2>✨ Story: {word.capitalize()}</h2>"),
                        widgets.HTML(f"<p style='font-size:16px'>{stories[word]}</p>"),
                        restart_button
                    ]))
                else:
                    display(widgets.VBox([
                        widgets.HTML("<h3>❌ Story Not Found</h3>"),
                        widgets.HTML("<p>Try: sun, moon, ocean</p>"),
                        restart_button
                    ]))
            else:
                question = game.filtered_questions[game.current_question]
                if question["type"] == "question":
                    options = [widgets.Button(
                        description=opt,
                        layout=widgets.Layout(width='auto', height='40px')
                    ) for opt in question["options"]]

                    for btn in options:
                        btn.on_click(handle_answer)

                    display(widgets.VBox([
                        widgets.HTML(f"<h2>❓ Question {game.current_question + 1}/{len(game.filtered_questions)}</h2>"),
                        widgets.HTML(f"<p style='font-size:18px'>{question['question']}</p>"),
                        widgets.VBox(options, layout=widgets.Layout(width='100%')),
                        widgets.HTML(f"<p>Score: {game.score}</p>"),
                        progress_bar
                    ]))
                else:
                    answer_input = widgets.Text(
                        placeholder='Type your answer...',
                        layout=widgets.Layout(width='50%')
                    )
                    submit_btn = widgets.Button(description="Submit Answer")

                    def on_submit(b):
                        check_answer(answer_input.value)

                    submit_btn.on_click(on_submit)

                    display(widgets.VBox([
                        widgets.HTML(f"<h2>🧩 Puzzle {game.current_question + 1}/{len(game.filtered_questions)}</h2>"),
                        widgets.HTML(f"<p style='font-size:18px'>{question['puzzle']}</p>"),
                        widgets.HBox([answer_input, submit_btn]),
                        widgets.HTML(f"<p>Score: {game.score}</p>"),
                        progress_bar
                    ]))

        elif game.game_state == "feedback":
            question = game.filtered_questions[game.current_question]
            is_correct = game.answered_questions[game.current_question]

            display(widgets.VBox([
                widgets.HTML(f"""
                    <div style='background-color: {'#e6f7e6' if is_correct else '#ffebee'};
                                padding: 15px; border-radius: 5px; margin: 10px 0'>
                    <h3>{'✅ Correct!' if is_correct else '❌ Incorrect'}</h3>
                    <p style='font-size:16px'>{question.get('explanation', '')}</p>
                    </div>
                """),
                widgets.HTML(f"<p>Current Score: <strong>{game.score}/{len(game.filtered_questions)}</strong></p>"),
                next_button if game.current_question < len(game.filtered_questions) - 1 else widgets.HTML(),
                progress_bar
            ]))

        elif game.game_state == "completed":
            badge = "Novice"
            if game.score == len(game.filtered_questions):
                badge = "Grand Master"
            elif game.score >= len(game.filtered_questions) * 0.75:
                badge = "Expert"
            elif game.score >= len(game.filtered_questions) * 0.5:
                badge = "Adept"

            display(widgets.VBox([
                widgets.HTML("<h1 style='text-align:center'>🏆 Quiz Complete!</h1>"),
                widgets.HTML(f"""
                    <div style='text-align:center'>
                    <p style='font-size:20px'>Your final score: <strong>{game.score}/{len(game.filtered_questions)}</strong></p>
                    <p style='font-size:24px'>🏅 <strong>{badge}</strong></p>
                    </div>
                """),
                restart_button
            ]))

# ===== Initialize Game =====
start_button.on_click(start_game)
next_button.on_click(next_question)
restart_button.on_click(restart_game)

# Display the initial setup screen
update_display()
display(output)

Output()